In [1]:
import requests
import json
import random
import time
from pathlib import Path

In [3]:
def load_config():
    config_path = Path('config_extract.json')
    if config_path.exists():
        with open(config_path) as f:
            return json.load(f)
    else:
        raise FileNotFoundError("Config file not found. Ensure 'config_extract.json' is present and correctly configured.")


In [4]:
def get_response_json(first_page, last_page, rescraping=False):
    config = load_config()
    url = config["url"]
    headers = config["headers"]
    payload_template = config["payload_template"]

    for page in range(first_page, last_page + 1):
        # Update payload with the current page number
        payload = payload_template.copy()
        payload["pagination"]["page"] = page
        payload = json.dumps(payload)

        response = requests.post(url, headers=headers, data=payload)

        if response.status_code != 200:
            print(f"Page {page} had an issue. Status code: {response.status_code}")
            continue
        else:
            print(f"\nPage: {page} | Status code: {response.status_code}\n")

        re = 're' if rescraping else ''
        with open(f'./01_{re}scraping_json_files/page_{page}.json', 'w') as fp:
            json.dump(response.json()['items'], fp)

        delay = random.uniform(3, 5)
        print(f"Waiting {delay:.2f} seconds before the next request.")
        time.sleep(delay)

    return "script completed"

In [ ]:
# get_response_json(1, 10000,rescraping=False)